<a href="https://colab.research.google.com/github/yantvrs/Conversor_de_numeros_complexos/blob/main/ProjetoFinal/codes/Projeto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.9/102.9 kB 590.8 kB/s eta 0:00:00


In [ ]:
import networkx as nx
import pandas as pd
import osmnx as ox

ox.settings.log_console = True
ox.__version__

'1.8.0'

In [ ]:
def get_cycleways_graph(query):
    # Configuração das tags úteis para a biblioteca OSMnx
    useful_tags = ox.settings.useful_tags_way + ['cycleway'] + ['bicycle']
    ox.utils.config(use_cache=True, log_console=True, useful_tags_way=useful_tags)

    # Baixa o grafo da rede de ciclovias
    G = ox.graph_from_place(query=query, network_type='bike', simplify=False, retain_all=True)

    # Lista para armazenar as arestas que não são ciclovias
    non_cyc = []

    # Itera sobre as arestas do grafo
    for u, v, k, d in G.edges(keys=True, data=True):
        bi = False
        if "bicycle" in d and d['bicycle'] == 'designated':
            bi = True

        if d['highway'] != 'cycleway' and 'cycleway' not in d and not bi:
            non_cyc.append((u, v, k))

    # Remove as arestas que não são ciclovias
    G.remove_edges_from(non_cyc)

    # Remove nós isolados do grafo
    G = ox.utils_graph.remove_isolated_nodes(G)

    # Remove os nós que não são utilizados nas arestas
    unused_nodes = [node for node, degree in dict(G.degree()).items() if degree == 0]
    G.remove_nodes_from(unused_nodes)

    # Simplifica o grafo
    G = ox.simplify_graph(G)

    return G

In [ ]:
place1 = "São Paulo, Brazil"
G1 = get_cycleways_graph(place1)

<ipython-input-3-06547187d4d1>:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.utils.config(use_cache=True, log_console=True, useful_tags_way=useful_tags)


In [ ]:
place2 = "Amsterdam, Netherlands"
G2 = get_cycleways_graph(place2)

<ipython-input-33-06547187d4d1>:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.utils.config(use_cache=True, log_console=True, useful_tags_way=useful_tags)


In [ ]:
place3 = "Berlin, Germany"
G3 = get_cycleways_graph(place3)

<ipython-input-33-06547187d4d1>:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.utils.config(use_cache=True, log_console=True, useful_tags_way=useful_tags)


#Análise Comparativa

##Density Based Stats

In [ ]:
#>>>>>>>>>>>>>>>>>>>>>> SÃO PAULO <<<<<<<<<<<<<<<<<<<<<<<<
gdf = ox.geocode_to_gdf(place1)
areaSP = ox.project_gdf(gdf).unary_union.area
stats = ox.basic_stats(G1, area=areaSP)
pd.Series(stats)

n                                                                            1511
m                                                                            1883
k_avg                                                                    2.492389
edge_length_total                                                      726264.321
edge_length_avg                                                        385.695338
streets_per_node_avg                                                     3.054931
streets_per_node_counts         {0: 0, 1: 136, 2: 75, 3: 943, 4: 287, 5: 68, 6...
streets_per_node_proportions    {0: 0.0, 1: 0.0900066181336863, 2: 0.049636002...
intersection_count                                                           1375
street_length_total                                                    441312.741
street_segment_count                                                         1195
street_length_avg                                                      369.299365
circuity_avg    

In [ ]:
#>>>>>>>>>>>>>>>>>>>>>> AMSTERDAM <<<<<<<<<<<<<<<<<<<<<<<
gdf = ox.geocode_to_gdf(place2)
areaAMS = ox.project_gdf(gdf).unary_union.area
stats = ox.basic_stats(G2, area=areaAMS)
pd.Series(stats)

n                                                                            6301
m                                                                           11445
k_avg                                                                    3.632757
edge_length_total                                                     1405839.129
edge_length_avg                                                        122.834349
streets_per_node_avg                                                     3.152992
streets_per_node_counts         {0: 0, 1: 166, 2: 543, 3: 3922, 4: 1536, 5: 10...
streets_per_node_proportions    {0: 0.0, 1: 0.026345024599269958, 2: 0.0861767...
intersection_count                                                           6135
street_length_total                                                    871407.268
street_segment_count                                                         7235
street_length_avg                                                      120.443299
circuity_avg    

In [ ]:
#>>>>>>>>>>>>>>>>>>>>>> BERLIN <<<<<<<<<<<<<<<<<<<<<<
gdf = ox.geocode_to_gdf(place3)
areaBER = ox.project_gdf(gdf).unary_union.area
stats = ox.basic_stats(G3, area=areaBER)
pd.Series(stats)

n                                                                            6439
m                                                                            8090
k_avg                                                                    2.512813
edge_length_total                                                     1650596.682
edge_length_avg                                                        204.029256
streets_per_node_avg                                                       2.9312
streets_per_node_counts         {0: 0, 1: 804, 2: 907, 3: 2955, 4: 1511, 5: 23...
streets_per_node_proportions    {0: 0.0, 1: 0.12486410933374748, 2: 0.14086038...
intersection_count                                                           5635
street_length_total                                                   1063512.025
street_segment_count                                                         5557
street_length_avg                                                      191.382405
circuity_avg    

## Outras métricas

In [ ]:
#CICLOVIA POR HABITANTE

populacaoSP = 11451245
populacaoAMS = 1603531
populacaoBER = 3711930

#Armazenando o comprimento (em metros) de ciclovias [retirados da função stats]
comprimento_ciclovias_SP = ox.stats.street_length_total(ox.utils_graph.get_undirected(G1))
comprimento_ciclovias_AMS = ox.stats.street_length_total(ox.utils_graph.get_undirected(G2))
comprimento_ciclovias_BER = ox.stats.street_length_total(ox.utils_graph.get_undirected(G3))


ciclovias_por_habitanteSP = comprimento_ciclovias_SP / populacaoSP
ciclovias_por_habitanteAMS = comprimento_ciclovias_AMS / populacaoAMS
ciclovias_por_habitanteBER = comprimento_ciclovias_BER / populacaoBER

print(f"Metros de ciclovias por habitante em São Paulo: {ciclovias_por_habitanteSP}")
print(f"Metros de ciclovias por habitante em Amsterdã: {ciclovias_por_habitanteAMS}")
print(f"Metros de ciclovias por habitante em Berlim: {ciclovias_por_habitanteBER}")



Metros de ciclovias por habitante em São Paulo: 0.03853840704657005
Metros de ciclovias por habitante em Amsterdã: 0.5434302598453048
Metros de ciclovias por habitante em Berlim: 0.28651187522394034


In [ ]:
# DENSIDADE DE CICLOVIAS

# Calcula a densidade de ciclovias
densidade_ciclovias_SP = comprimento_ciclovias_SP / areaSP * 1000000
print(f"Densidade de ciclovias em São Paulo: {densidade_ciclovias_SP} metros por quilômetro quadrado")

densidade_ciclovias_AMS = comprimento_ciclovias_AMS / areaAMS * 1000000
print(f"Densidade de ciclovias em Amesterdã: {densidade_ciclovias_AMS} metros por quilômetro quadrado")

densidade_ciclovias_BER = comprimento_ciclovias_BER / areaBER * 1000000
print(f"Densidade de ciclovias Berlim: {densidade_ciclovias_BER} metros por quilômetro quadrado")

Densidade de ciclovias em São Paulo: 289.8554099103693 metros por quilômetro quadrado
Densidade de ciclovias em Amesterdã: 3971.441388233188 metros por quilômetro quadrado
Densidade de ciclovias Berlim: 1194.0311446996689 metros por quilômetro quadrado


# Identificação de Áreas Críticas